In [1]:
# Goal: Generate content for music inspired cocktails
template = {
    "title": "", # get list of song names. find rhymes for each word. replace closest word to alcohol. 
    "description": "", # tracery grammar 
    "brewed_date": "", # date of song
    "style": "", # music genre
    "alcohol_content": "", # random num?
    "notes": "", # list of misc liqeurs
    "pairing": "", # find list of song names. find rhymes for each word, replace closest word to X food
    "serving_suggestion": "" # tracery grammar using words from lyrics (closest words to temperature etc.)
}

In [6]:
# Download data, shell script
# song lyrics
! wget -O scratch_data/lyrics.csv \
https://raw.githubusercontent.com/walkerkq/musiclyrics/master/billboard_lyrics_1964-2015.csv 

--2017-09-24 10:08:05--  https://raw.githubusercontent.com/walkerkq/musiclyrics/master/billboard_lyrics_1964-2015.csv
Resolving raw.githubusercontent.com... 151.101.164.133
Connecting to raw.githubusercontent.com|151.101.164.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7953541 (7.6M) [text/plain]
Saving to: ‘scratch_data/lyrics.csv’

scratch_data/lyrics 100%[===================>]   7.58M  --.-KB/s    in 0.1s    

2017-09-24 10:08:05 (51.7 MB/s) - ‘scratch_data/lyrics.csv’ saved [7953541/7953541]



In [48]:
import csv
lyrics_raw = csv.DictReader(open('scratch_data/lyrics.csv',  encoding='ISO-8859-1'))
lyrics = []
for row in lyrics_raw:
    # Keys: Rank, Song, Artist, Year, Lyrics, Source
    not_obscure = int(row['Year']) >= 2010 or (int(row['Rank']) <= 5 and int(row['Year']) >= 1994)
    if (not_obscure) :
        lyrics.append({
            'rank': row['Rank'],
            'song': row['Song'],  
            'artist': row['Artist'],
            'year': row['Year'],
            'lyrics': row['Lyrics'],
            'source': row['Source']
        });

print(len(lyrics))

680
